# Required Imports

In [1]:
import sys, random, math, pickle
from time import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import MSELoss
from tensorboard import default
import torch.nn.functional as F
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
sys.path.append('DG/gan')
import gc
print(device)

cuda:1


# Import Data (X,Y) and Mask

In [2]:
data = np.load('data/google/data_train.npz')

data_feature = torch.from_numpy(data['data_feature']).float() #[50000, 2500, 9]
data_attribute = torch.from_numpy(data['data_attribute']) #[50000,4]
data_gen_flag = torch.from_numpy(data['data_gen_flag'])   # (50000, 2500)

# Generation Flag

In [3]:
def get_generation_flags(mask_matrix):
    P1 = (mask_matrix == 1)
    P2 = (mask_matrix == 0)
    for p1,p2 in zip(P1,P2):
        idx_to_be_0_1 = np.where(p1==True)[0][-1]
        p1[idx_to_be_0_1]= False
        p2[idx_to_be_0_1] = True
        
        p1[idx_to_be_0_1+1:] = False
        p2[idx_to_be_0_1+1:] = False 
    res = torch.cat((P1.unsqueeze(2),P2.unsqueeze(2)),2)
    return res.long()

In [4]:
gen_flag = get_generation_flags(data_gen_flag) # [6529, 2500, 2]
data_feature = torch.cat((data_feature,gen_flag),2) # [6529, 2500, 11]

In [5]:
data_feature.shape

torch.Size([50000, 2500, 11])

# Save Data

In [6]:
np.savez('data_train', data_feature=data_feature,data_attribute=data_attribute,data_gen_flag=data_gen_flag )